# Assigment 4

In [1]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
def align_field(V, F, TT, hard_id, hard_value):
    assert(hard_id[0] > 0)
    assert(hard_id.shape[0] == hard_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    index = []
    data = []
    ii = []
    jj = []
    
    for f in range(F.shape[0]):  
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()      
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef * ef.conjugate());  ii.append(f); jj.append(f)
            data.append(-ef * eg.conjugate()); ii.append(f); jj.append(g)
            data.append(-ef.conjugate() * eg); ii.append(g); jj.append(f)
            data.append(-eg * eg.conjugate()); ii.append(g); jj.append(g)
        
    # Rhs of the system
     
    
    # Convert the constraints into the complex polynomial coefficients and add them as hard constraints


    return R

In [4]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [5]:
R = align_field(v, f, tt, cf, c)
plot_mesh_field(v, f, R, cf)

D:\Anaconda\envs\GM\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
D:\Anaconda\envs\GM\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…